# Time Series Project
## CodeUp Data Science | April 2022

#### For this project you will be analyzing data from a superstore. The project is open ended and you can choose the direction you want to take it based on the paths outlined below.


<hr>

## SPEC:

__VP of Product:__ Which product line should we expand?

    - Is there a product category that is particularly profitable for us? Does one or another stand out in terms of sales volume? Does this vary by customer segment?



### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import Stephen's files
import acquire_stephen 
import prepare_stephen 

# Import jesse's files
import acquire_jesse
import prepare_jesse


ModuleNotFoundError: No module named 'acquire_stephen'

In [ ]:
df = prepare_superstore_data